# Face-Tracker Project

Is it possible to track a person and their movement with a moving camera? I plan on exploring this with a raspberry pi and moving camera, and possibly exploring the idea of tracking a toddler (my son).

---

## Table of Contents
 - [EDA and Cleaning](#EDA-and-Cleaning)
   - [Planning](#Planning)
   - [Collecting photos](#Collecting-photos)
 - [Setting up Hardware](#Setting-up-Hardware)
   - [Hardware list](#Hardware-list)
   - [Pimoroni Pan-Tilt HAT](#Pimoroni-Pan-Tilt-HAT)
   - [Raspberry Pi](#Raspberry-Pi)
   - [Intel Neural Compute Stick 2 (Movidius)](#Intel-Neural-Compute-Stick-2-(Movidius))
 - [Setting up Software](#Setting-up-Software)
   - [Installing Raspberry Pi OS](#Installing-Raspberry-Pi-OS)
     - [Flashing with balena-Etcher](#Flashing-with-balena-Etcher)
   - [Setting up the Raspberry Pi](#Setting-up-the-Raspberry-Pi)
   - [Installing OpenCV](#Installing-OpenCV)
   - [Installing pantilthat repository from Pimoroni](#Installing-pantilthat-repository-from-Pimoroni)
   - [Installing OpenVINO](#Installing-OpenVINO)
   - [Neural Compute SDK v2 (NCSDK2)](#Neural-Compute-SDK-v2-(NCSDK2))
 - [Testing Hardware with Software](#Testing-Hardware-with-Software)
   - [Camera Testing](#Camera-Testing)
   - [Pan Tilt Hat testing](#Pan-Tilt-Hat-testing)
   - [Haar Cascade (from OpenCV package)](#Haar-Cascade-(from-OpenCV-package))
   - [YOLOv3 by xiaochus on github](#YOLOv3-by-xiaochus-on-github)
   - [Putting it all together](#Putting-it-all-together)
 - [Preprocessing](#Preprocessing)
   - [Annotating with CVAT](#Annotating-with-CVAT)
   - [Annotating with RoboFlow](#Annotating-with-RoboFlow)
   - [Generating Augmented Images with RoboFlow](#Generating-Augmented-Images-with-RoboFlow)
 - [Training](#Training)
   - [Generate Weights - Add Colab notebooks](#Generate-Weights---Add-Colab-notebooks)
   - [Adding weights, names, and cfg to the code](#Adding-weights,-names,-and-cfg-to-the-code)
 - [Analysis](#Analysis)
   - [You Only Look Once (YOLO)](#You-Only-Look-Once-(YOLO))
   - [Haar Cascade: from 2001 yet still effective](#Haar-Cascade:-from-2001-yet-still-effective)
   - [Performance of YOLO vs Haar Cascade](#Performance-of-YOLO-vs-Haar-Cascade)
      - [Performance on Pi vs Laptop](#Performance-on-Pi-vs-Laptop)
   - [Look mAP (mean Average Performance)](#Look-mAP-(mean-Average-Performance))
 - [Conclusion](#Conclusion)
 - [Future Considerations and Recommendations](#Future-Considerations-and-Recommendations)
   - [install Ubuntu on Pi](#install-Ubuntu-on-Pi)
      - [try to get Movidius to work](#try-to-get-Movidius-to-work)
   - [Implement NN on that HW or better HW](#Implement-NN-on-that-HW-or-better-HW)

---

---
## EDA and Cleaning
[(back to top)](#Face-Tracker-Project)

---

### Planning
[(back to top)](#Face-Tracker-Project)

This project was inspired by my son during video calls with his grandparents in Florida (my mom and dad). He is an active little fellow, and I thought it would be neat if there was a way that a camera could track his movement and adjust position so my son would generally stay in frame. This way my hands would be free to play with him (instead of holding the phone just out of his reach).


Since my phone does not have the capability to move on its own, I thought the next best thing would be a Raspberry Pi! For those who do not know, a [Raspberry Pi](https://www.raspberrypi.org/products/raspberry-pi-4-model-b/) is not only a delicious dessert, but it is also a tiny inexpensive computer, often described as "about the size of a credit card". When talking about the circuit board, this is accurate, but with all the components attached to it, and then put into a case, the size is closer to "about the size of a 2-inch thick credit card". 


I purchased this [CanaKit Raspberry Pi 4](https://www.amazon.com/gp/product/B08956GVXN/ref=ppx_yo_dt_b_asin_title_o06_s00?ie=UTF8&psc=1) which boasts a 4-core processor and a whopping 8GB of RAM (where the previous Raspberry Pi a nd 3 only had 1GB of RAM). This version of the Raspberry pi utilizes a 64-bit CPU which allows use for a 64-bit OS, but with some caveats.


From past research I know the Raspberry Pi has camera capabilities, as well as something called a "Pan-Tilt-Hat" which the camera can be mounted to for movement. The "Pan-Tilt-Hat" is a circuit board that can sit atop the Raspberry Pi (like a hat, hence that part of the name), and it contains two servos that allow the device to pan horizontally, and tilt vertically. 


From my research for this project, I found that I would need to install something called "OpenCV" which is a C/C++ based library that has a port to the Python programming language. This library seems to be __the__ library for anything involving object tracking and/or face detection.

---

### Collecting photos
[(back to top)](#Face-Tracker-Project)

With some help, I was able to download some training photos from Northeastern University's Augmented Cognition Lab's website. I trained a Keras model with these photos using Transfer Learning, but I later decided to abandon this route when I realized that I likely needed to freeze my Keras model in order to load it with OpenCV; A technique I am currently unfamiliar with. While this model scored well with the test set, I thought it may be the best to go another route since my son is now walking/running, and most of these images were infants crawling and laying down.

With this in mind, I decided to instead use photos of my son that my wife and I had taken over the past year. These were easy to gather and were more accessible; and I thoroughly enjoyed looking through these photos to find a good set for annotating, training, testing, and validating. It was around this time I began to discover that there are many, many methods of saving, loading, and keeping neural net data, which typically involves a weights file and another file for the network architecture.

---

---

## Setting up Hardware
[(back to top)](#Face-Tracker-Project)

---

<img src="./assets/img/hardware_pic.png" width="400" />

### Hardware list
 - [Raspberry Pi 4, 8GB, CanaKit](https://www.amazon.com/gp/product/B08956GVXN/ref=ppx_yo_dt_b_asin_title_o06_s00?ie=UTF8&psc=1)
 - [Pimoroni Pan-Tilt HAT full kit](https://www.amazon.com/gp/product/B01N5E3I3W/ref=ppx_yo_dt_b_asin_title_o05_s00?ie=UTF8&psc=1)
 - [Raspberry Pi Camera Module V2, 8MP](https://www.amazon.com/gp/product/B01ER2SKFS/ref=ppx_yo_dt_b_asin_title_o06_s01?ie=UTF8&psc=1)
 - [12" Adafruit Flex Cable for Raspberry Pi Camera](https://www.amazon.com/gp/product/B00I6LJ19G/ref=ppx_yo_dt_b_asin_title_o06_s00?ie=UTF8&psc=1)
 - [Intel Neural Compute Stick 2 (Movidius)](https://www.amazon.com/gp/product/B07KT6361R/ref=ppx_yo_dt_b_asin_title_o06_s00?ie=UTF8&psc=1)

---

<br /><br /><img src="./assets/img/pth-assembled.png" width="200" />

### Pimoroni Pan-Tilt HAT
[(back to top)](#Face-Tracker-Project)

I wrote a [blog post](https://christopher-j-caldarella.medium.com/assembling-the-pan-tilt-hat-ae90fc75d47c) about how I assembled the hardware for the Pimoroni Pan-Tilt Hat. Hopefully someone else finds it helpful. In summation, I bolted on the mechanism with the servos to the Pan-Tilt PCB, plugged the servos into the PCB, attached a face plate to the camera, and affixed the camera to the Pan-Tilt mechanism. Then I connected the Camera to the Raspberry Pi, and then the Pan-Tilt HAT. My [blog post](https://christopher-j-caldarella.medium.com/assembling-the-pan-tilt-hat-ae90fc75d47c) that I had mentioned earlier has pictures and goes into further detail.

---

<br /><br /><img src="./assets/img/raspi.png" width="200" />

### Raspberry Pi
[(back to top)](#Face-Tracker-Project)

The Raspberry Pi did not require much assembly once the Pan-Tilt HAT was attached, especially since the case does not fit on the Raspberry Pi with the Pan-Tilt mechanism protruding from the top of my Raspberry Pi. ALl that was left was to insert the microSD card (the system's Solid-State-Drive), plug in a USB Mouse and keyboard, plug in the monitor, and plug in the power. Fortunately, the CanaKit provided the microSD card, the Monitor cable, and the power supply.

---

<br /><br /><img src="./assets/img/movidius.png" width="200" />

### Intel Neural Compute Stick 2 (Movidius)
[(back to top)](#Face-Tracker-Project)

I had trouble getting this to work, and I talk more about it in the next section, [Setting up Software](#Setting-up-Software). Initially I was trying to get the [Google Coral USB *Accelerator*](https://coral.ai/docs/accelerator/get-started/), not to be confused with the very similar looking \$200 Coral compute *device*. 

The Google Coral USB Accelerator appears to be much more user-firendly, especially for Linux-based machines. But everywhere I looked these devices were back-ordered or sold-out, and I did not see any that would arrive in a timely fashion.

---

---

## Setting up Software
[(back to top)](#Face-Tracker-Project)

---

<img src="./assets/img/Raspberry_Pi_OS_Logo.png" width="400" />

### Installing Raspberry Pi OS
[(back to top)](#Face-Tracker-Project)

The CanaKit I purchased came with a microSD card preloaded with "NOOBS" or "New Out Of Box Sofware". NOOBS starts with GUI and you can pick which OS you want to install, but after some reasearch I found that NOOBS is deprecated in favor of the Raspberry Pi OS (formerly known as Raspbian). Reasearching the best experieince with running OpenCV on Raspberry Pi, I decided to use the latest Raspbian OS.

The [Raspberry Pi Foundation](https://www.raspberrypi.org/about/) (a UK registered Charity Organization) makes it very easy to [install your Raspberry Pi OS](https://www.youtube.com/watch?v=ntaXWS8Lk34). You can download the Operating System image directly and flash your microSD card with Raspberry Pi's software or a software of your choice. I already had Balena Etcher, another application that can flash microSD cards, on my computer so I used that software instead of the Raspberry Pi's "Imager" executable. 


<img src="./assets/img/balena_etcher_logo.png" width="400" />

#### Flashing with balena-Etcher
[(back to top)](#Face-Tracker-Project)

Flashing a microSD card is a very process, although it may sound intimidating. Typically this process is used for installing a Linux Operating System of one form or another, so you start by search for the Operating System of your choice. For instance, when I type "Linux Image" into Google, the first thing I see is a link to download Ubuntu, a version of Linux. Clicking on the link brings me to a page that has a prominent "Download" button.

<img src="./assets/img/balena-etcher.png" width="400" />

In this case, I searched the [Raspberry Pi website's Software/Operating System](https://www.raspberrypi.org/software/operating-systems/) page for a Raspberry Pi Operating System compatible with my Raspberry Pi. At this point, I plugged in my microSD card. After downloading the software, I ran [Balena Etcher](https://www.balena.io/etcher/), selected the image I had just downloaded, selected the microSD card as my "Drive", and clicked "Flash", and ten minutes later my "Solid-State-Drive" for my Raspberry Pi was ready. I later used the Raspberry Pi's "Imager" software and it is just as easy to use.


---

<img src="./assets/img/raspi-config.png" width="600" />

### Setting up the Raspberry Pi
[(back to top)](#Face-Tracker-Project)

It is worth mentioning that the Raspberry Pi has configurations for many things, including the camera, and devices like the Pan-Tilt HAT, but these things are disabled and need to be enabled. Raspberry pi has a useful and intuitive COnfiguration setup (both a GUI and in the terminal). By running `raspi-config` I was abel to easily navigate the menu and turn on these things, as well as VNC, and SSH.

---

<img src="./assets/img/opencv_logo.png" width="400" />

### Installing OpenCV
[(back to top)](#Face-Tracker-Project)

Although flashing a microSD card is very easy, it something I do not like to perform on the same card multiple times. Since microSD cards are a solid state drive device, they have limited read and write cycles. If anyone remembers listening to cassette tapes or watching VHS tapes, it is similar to how after much use your favorite cassette or VHS tape would start to degrade and not sound or look as good. However in this case, the microSD card just fails to work anymore.

With this in mind I wanted to be very careful with my process. I later bought 5 more microSD cards just to be safe. I then used a tool named [Win32 Disk Imager](https://win32diskimager.download/) to make a backup image of my work in case my microSD card failed, so I had a backup and so I could quickly image a new card. Although I never had to use this, this would be very helpful since the OpenCV installation can take a while. In general, this is a good solution for backing up your environments; I was already using version control with Git to keeep all of my important work updated.

I uesd the following guide to help me install OpenCV on Raspbian Buster:
https://www.pyimagesearch.com/2019/09/16/install-opencv-4-on-raspberry-pi-4-and-raspbian-buster/

What was nice about this tutorial is the fact that it guided you to use Python's virtual environment module, venv. I also ended up installing miniconda on the Raspberry Pi, but I never used the two modules together. So I ended up installing OpenCV and its dependencies twice, once for my "cv" `venv` environment and again for my "opencv" `conda` environment. In the end, my "opencv" `conda` environment waseasier to use with the Python files I was using and writing. But the "cv" `venv` environment seemed to work better with the Intel Compute Stick I tried to deploy.

---

<img src="./assets/img/pimoroni_logo.png" width="400" />

### Installing pantilthat repository from Pimoroni
[(back to top)](#Face-Tracker-Project)

This is a simple and straightforward process where you use curl to download and install the library (or apt-get).

I also downloaded the python-smbus library as well as the Pimoroni example, PanTilt-FaceTracker.

---

<img src="./assets/img/openvino-logo.png" width="400" />

### Installing OpenVINO
[(back to top)](#Face-Tracker-Project)

OpenVINO is the software that needs to be inplace to use the Intel Neural Compute Stick 2. I wrote a [blog post](https://christopher-j-caldarella.medium.com/installing-the-openvino-toolkit-250f089e4e32) outlining the process I took to install this software. I found the need to do this so I could properly wrap my head around the process since most of the guides that were around were from 2019, and had a slightly different file structure, not to mention that the guides have seemingly inconsistent file paths. 

There is a youtube video by Intel that has our GUide running long commands, shown in the background, and our Guide just reads the base command, e.g. "*run wget and un-tar*", and then the video cuts to another screen to go through the next step, with no links in the video description. The video description does, however, include the OpenVINO guide, which I had already explored, and halfway through decides that our installation directory is no longer "openvino" but now "openvino_2021". 

Both the video and the guide fail to give a properly running example since neither provide the path to the files it is using for the example. I was able to get the example to run by exploring every single folder, and there are both files and folders with the same or very similar names.

---

<img src="./assets/img/ncs2-lid-box.png" width="400" />

### Neural Compute SDK v2 (NCSDK2)
[(back to top)](#Face-Tracker-Project)

I attempted for many days to install Intels [NCSDK 2 repository](https://github.com/movidius/ncsdk) on my Raspberry Pi with little success. I ran into a minor hiccup with the `install.sh` file where it looks for a Debian/Raspbian version greater than 81, 90, and 91; However Debian and Raspbian versions are 8.1, 9.0, and 9.1 respectively (Stretch, Jessie, and Jessie). So I manually changed this in the file to look for versions greater than 8 and 9 since it must be an integer value and since I am on version 10 (Buster).

After getting aroundthis, I ran through all sorts of issues trying to get teh NCSDK2 `make install` command to work; It errors out while trying to install Tensorflow v1.11. I manually installed Tensorflow in the virtual environment, once upt o version 1.11, and again up to version 1.14, but installation still fails at installing Tensorflow, even though therequirement is already fulfiled.

This was after pouring over a littany of error messages, one of which told me: `“python setup.py egg_info” failed with error code 1`

Looking for solutions to this error, I eventually got past it only to be met by new errors. I believe I got past the "egg_info" error by updating and upgrading the OS again as well as upgrading pip3.

Getting the Intel Neural Compute stick to work with the Raspberry Pi is something I am continuing to work on. I will talk more about this later in the [Future Considerations and Recommendations](#Future-Considerations-and-Recommendations) section.

---

---

## Testing Hardware with Software
[(back to top)](#Face-Tracker-Project)

Trying out each facet mentioned above and trying to put it all together

---

### Camera Testing
[(back to top)](#Face-Tracker-Project)

I ran the following command and captured a still. It came out upside down because of how the Pan-Tilt HAT is oriented. The Pan-Tilt HAT software also has a line that flips the picture to correct this.

`raspistill -o ~/Pictures/new_image.jpeg`

---

### Pan Tilt Hat testing
[(back to top)](#Face-Tracker-Project)

Pan-Tilt-Hat-testing

#### Issues with packaged pan/tilt
[(back to top)](#Face-Tracker-Project)

Issues-with-packaged-pan/tilt

---

### Haar Cascade (from OpenCV package)
[(back to top)](#Face-Tracker-Project)

Haar-Cascade-(from-OpenCV-package)

---

### YOLOv3 by xiaochus on github
[(back to top)](#Face-Tracker-Project)

YOLOv3-by-xiaochus-on-github)

---
### Putting it all together
[(back to top)](#Face-Tracker-Project)

Putting-it-all-together)

---

---

## Preprocessing
[(back to top)](#Face-Tracker-Project)

Preprocessing

---

### Annotating with CVAT
[(back to top)](#Face-Tracker-Project)

Annotating-with-CVAT

---

### Annotating with RoboFlow
[(back to top)](#Face-Tracker-Project)

Annotating-with-RoboFlow

---

### Generating Augmented Images with RoboFlow
[(back to top)](#Face-Tracker-Project)

Generating-Augmented-Images-with-RoboFlow

---

---

## Training
[(back to top)](#Face-Tracker-Project)

Training

---

### Generate Weights - Add Colab notebooks
[(back to top)](#Face-Tracker-Project)

Generate-Weights---Add-Colab-notebooks

---

### Adding weights, names, and cfg to the code
[(back to top)](#Face-Tracker-Project)

Adding-weights,-names,-and-cfg-to-the-code

---

---

## Analysis
[(back to top)](#Face-Tracker-Project)

Analysis

---

### You Only Look Once (YOLO)
[(back to top)](#Face-Tracker-Project)

You-Only-Look-Once-(YOLO)

---

### Haar Cascade: from 2001 yet still effective
[(back to top)](#Face-Tracker-Project)

Haar-Cascade:-from-2001-but-still-effective

---

### Performance of YOLO vs Haar Cascade
[(back to top)](#Face-Tracker-Project)

Performance-of-YOLO-vs-Haar-Cascade

#### Performance on Pi vs Laptop
[(back to top)](#Face-Tracker-Project)

Performance-on-Pi-vs-Laptop

---
### Look mAP (mean Average Performance)
[(back to top)](#Face-Tracker-Project)

Look-mAP-(mean-Average-Performance)

---

---

Conclusion
[(back to top)](#Face-Tracker-Project)

Conclusion

---

---

## Future Considerations and Recommendations
[(back to top)](#Face-Tracker-Project)

Future-Considerations-and-Recommendations

---

### install Ubuntu on Pi
[(back to top)](#Face-Tracker-Project)

install-Ubuntu-on-Pi

#### try to get Movidius to work
[(back to top)](#Face-Tracker-Project)

try-to-get-Movidius-to-work

---
### Implement NN on that HW or better HW
[(back to top)](#Face-Tracker-Project)

Implement-NN-on-that-HW-or-better-HW

---